In [1]:
from pathlib import Path
import utils.sonar_image_analysis as iau
import importlib
importlib.reload(iau)

# Centralized sonar defaults (inserted by sweep)
from utils.sonar_config import SONAR_VIS_DEFAULTS
sonar_config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = sonar_config['range_min_m']
RANGE_MAX_M = sonar_config['range_max_m']
DISPLAY_RANGE_MAX_M = sonar_config['display_range_max_m']

# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

In [2]:


# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename 
# '2024-08-22_14-47-39'
# '2024-08-22_14-29-05'
# '2024-08-20_14-22-12'
# '2024-08-20_14-31-29' HARD
# '2024-08-20_18-47-40' EASY
TARGET_BAG = '2024-08-20_14-31-29'  # change this to your desired bag ID
from utils.sonar_config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)
print(f'Selected NPZ file: {selected.name} (index={NPZ_FILE_INDEX})')

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data
Selected NPZ file: 2024-08-20_14-31-29_data_cones.npz (index=8)


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [3]:
# Create the unified processor
processor = iau.SonarDataProcessor()

# Use the enhanced video creation with CORE processor
video_path = iau.create_enhanced_contour_detection_video_with_processor(
    npz_file_index=NPZ_FILE_INDEX,          # Which NPZ file to use
    frame_start=1,           # Starting frame
    frame_count=1500,         # Number of frames to process
    frame_step=1,            # Step between frames
    output_path=Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / 'core_aoi_tracking.mp4',
    processor=processor      # Use our CORE processor
)

=== ENHANCED VIDEO CREATION (Simplified) ===
Creating video with simplified processor...
Frames: 1500, step: 1


Processing 1059 frames with simplified processor...
Processed 10/1059 frames
Processed 20/1059 frames
Processed 30/1059 frames
Processed 40/1059 frames
Processed 50/1059 frames
Processed 60/1059 frames
Processed 70/1059 frames
Processed 80/1059 frames
Processed 90/1059 frames
Processed 100/1059 frames
Processed 110/1059 frames
Processed 120/1059 frames
Processed 130/1059 frames
Processed 140/1059 frames
Processed 150/1059 frames
Processed 160/1059 frames
Processed 170/1059 frames
Processed 180/1059 frames
Processed 190/1059 frames
Processed 200/1059 frames
Processed 210/1059 frames
Processed 220/1059 frames
Processed 230/1059 frames
Processed 240/1059 frames
Processed 250/1059 frames
Processed 260/1059 frames
Processed 270/1059 frames
Processed 280/1059 frames
Processed 290/1059 frames
Processed 300/1059 frames
Processed 310/1059 frames
Processed 320/1059 frames
Processed 330/1059 frames
Processed 340/1059 frames
Processed 350/1059 frames
Processed 360/1059 frames
Processed 370/1059 fr

In [4]:
engine = iau.DistanceAnalysisEngine()
distance_results = engine.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,        # Start from frame 1
    frame_count=1500,      # Analyze all frames from the video
    frame_step=1          # Every frame
)

print(f"Analyzed {len(distance_results)} frames using elliptical AOI")
print(f" Detection success rate: {distance_results['detection_success'].mean():.1%}")
print(f" Elliptical tracking active: {(distance_results['tracking_status'].str.contains('ELLIPSE')).sum()} frames")

=== DISTANCE ANALYSIS FROM NPZ ===
Analyzing: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_data_cones.npz
Processing 1059 frames from 1 (step=1)
  Processed 50/1059 frames (Success rate: 100.0%)
  Processed 100/1059 frames (Success rate: 100.0%)
  Processed 150/1059 frames (Success rate: 100.0%)
  Processed 200/1059 frames (Success rate: 100.0%)
  Processed 250/1059 frames (Success rate: 100.0%)
  Processed 300/1059 frames (Success rate: 100.0%)
  Processed 350/1059 frames (Success rate: 100.0%)
  Processed 400/1059 frames (Success rate: 100.0%)
  Processed 450/1059 frames (Success rate: 100.0%)
  Processed 500/1059 frames (Success rate: 100.0%)
  Processed 550/1059 frames (Success rate: 100.0%)
  Processed 600/1059 frames (Success rate: 100.0%)
  Processed 650/1059 frames (Success rate: 100.0%)
  Processed 700/1059 frames (Success rate: 100.0%)
  Processed 750/1059 frames (Success rate: 100.0%)
  Processed 800/1059 frames (Success rate: 100.0%)
  Processed 850/1059 frame

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [5]:
# Auto-detect pixel->meter mapping from the selected NPZ (if available)
try:
    cones, ts, extent, meta = iau.load_cone_run_npz(selected)
    T, H, W = cones.shape
    x_min, x_max, y_min, y_max = extent
    width_m = float(x_max - x_min)
    height_m = float(y_max - y_min)
    px2m_x = width_m / float(W)
    px2m_y = height_m / float(H)
    pixels_to_meters_avg = 0.5 * (px2m_x + px2m_y)
    image_shape = (H, W)
    sonar_coverage_meters = max(width_m, height_m)
    print(f"Detected NPZ extent: x=[{x_min:.3f},{x_max:.3f}] m, y=[{y_min:.3f},{y_max:.3f}] m")
    print(f"Image shape from NPZ: H={H}, W={W}")
    print(f"meters/pixel: x={px2m_x:.6f}, y={px2m_y:.6f}, avg={pixels_to_meters_avg:.6f}")
except Exception as e:
    print("Could not read NPZ metadata:", e)
    print("Falling back to defaults from sonar_config.")
    from utils.sonar_config import CONE_H_DEFAULT, CONE_W_DEFAULT, DISPLAY_RANGE_MAX_M_DEFAULT
    image_shape = (CONE_H_DEFAULT, CONE_W_DEFAULT)
    sonar_coverage_meters = DISPLAY_RANGE_MAX_M_DEFAULT * 2  # approximate
    pixels_to_meters_avg = sonar_coverage_meters / max(image_shape)
    
# Use the computed pixels_to_meters_avg in downstream analysis
print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")

Detected NPZ extent: x=[-8.660,8.660] m, y=[0.000,10.000] m
Image shape from NPZ: H=700, W=900
meters/pixel: x=0.019245, y=0.014286, avg=0.016765
Using pixels_to_meters_avg = 0.016765 m/px


In [6]:
iau.VisualizationEngine.plot_distance_analysis(distance_results, "Real-World Distance Analysis")

In [7]:
# COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.net_distance_analysis as sda

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.sonar_config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\n RAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"    {key}: {len(data)} records")
    else:
        print(f"    {key}: None")

print(f"\n DISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"    {key}: {data_len} records - {info['description']}")

 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_14-31-29
📡 1. Loading Navigation Data...
    Loaded 559 navigation records
📡 2. Loading Guidance Data...
    Loaded 549 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
📡 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 33 USBL records
📡 5. Loading DVL Position...
    Loaded 283 DVL position records
 6. Loading Navigation Position...
    Loaded 278 navigation position records
📡 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_14-31-29
    Raw data loaded: 2/2
    Distance measurements: 4

 RAW DATA LOADED:
    navigation: 559 records
    guidance: 549 records

 DISTANCE MEASUREMENTS LOADED:
    USBL_3D: 33 records - 3D acoustic position
    USBL_Depth: 33 records - USBL depth measurement
    DVL_Position: 283 records - 3D DVL position
    Nav_Position: 278 records - 2D navigation position


In [8]:
fig, comparison_stats = iau.ComparisonEngine.compare_sonar_vs_dvl(distance_results, raw_data, sonar_coverage_m=sonar_coverage_meters, sonar_image_size=image_shape[0], use_plotly=True)
fig.show()


SONAR vs DVL COMPARISON STATISTICS:
Sonar mean distance: 2.020 m
DVL mean distance:   1.968 m
Scale ratio (Sonar/DVL): 1.027x
Sonar duration: 67.1s (1059 frames)
DVL duration:   67.0s (559 records)


/Users/eirikvarnes/code/SOLAQUA/utils/sonar_image_analysis.py:311: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/eirikvarnes/code/SOLAQUA/utils/sonar_image_analysis.py:318: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [9]:
# Generate Video with Both DVL and Sonar Analysis Overlays (Improved Sync)
# ==========================================================================
import utils.sonar_and_foto_generation as sg

# Enable automatic video detection with improved synchronization
from utils.sonar_config import VIDEO_CONFIG
VIDEO_CONFIG['enable_video_overlay'] = True  
VIDEO_CONFIG['max_sync_tolerance_seconds'] = 1.0  # Tighter sync tolerance

print("GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS")
print("=" * 60)
print(f" Target Bag: {TARGET_BAG}")
print(f" DVL Data: {len(raw_data.get('navigation', []))} records")
print(f" Sonar Analysis: {len(distance_results)} frames")
print(f" Auto-detect video: enabled (PNG frames + index.csv)")
print(f"⏱  Sync tolerance: {VIDEO_CONFIG['max_sync_tolerance_seconds']}s")

# Generate the video with both overlays and improved sync
try:
    video_path = sg.export_optimized_sonar_video(
        TARGET_BAG=TARGET_BAG,
        EXPORTS_FOLDER=EXPORTS_FOLDER,
        START_IDX=1,
        END_IDX=1200,  
        STRIDE=1,    
        AUTO_DETECT_VIDEO=True,       # Enable automatic video file detection
        INCLUDE_NET=True,             # Enable DVL net distance overlay
        SONAR_DISTANCE_RESULTS=distance_results,  # Enable sonar analysis overlay
        NET_DISTANCE_TOLERANCE=0.5,
        NET_PITCH_TOLERANCE=2.0,      # Increased from 0.3 to 2.0 seconds for better pitch sync
    )

    print(f"\n Video generated successfully!")
    print(f" Output: {video_path}")
    print(f"\n Check the video output:")
    print(f"   - Camera frames should sync properly with sonar")
    print(f"   - When camera data ends, video should switch to sonar-only")
    print(f"   - No frozen camera frames should appear")
    
except Exception as e:
    print(f"Error generating video: {e}")
    import traceback
    traceback.print_exc()
    print("\n Troubleshooting:")
    print("  1. Run the video detection cell above to verify files exist")
    print("  2. Check that PNG frames directory has index.csv file")
    print("  3. Try disabling video overlay: VIDEO_CONFIG['enable_video_overlay'] = False")
    print("  4. Check exports directory structure")

GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS
 Target Bag: 2024-08-20_14-31-29
 DVL Data: 559 records
 Sonar Analysis: 1059 frames
 Auto-detect video: enabled (PNG frames + index.csv)
⏱  Sync tolerance: 1.0s
OPTIMIZED SONAR VIDEO
Target Bag: 2024-08-20_14-31-29
   Cone Size: 900x700
   Range: 0.0-5.0m | FOV: 120.0°
Auto-detecting video files for bag: 2024-08-20_14-31-29
Found video frames: 2024-08-20_14-31-29_video__image_compressed_image_data_frames
Using frames directory: /Volumes/LaCie/SOLAQUA/exports/frames/2024-08-20_14-31-29_video__image_compressed_image_data_frames
Camera: enabled
Net-line: enabled (dist tol=0.5s, pitch tol=2.0s)
Sonar Analysis: enabled
   Loading sonar data: sensor_sonoptix_echo_image__2024-08-20_14-31-29_video.csv
Loaded 1060 sonar frames in 42.29s
Loaded 559 navigation records in 0.02s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
Loaded 844 camera index entries
   Frames: 1..1059 (step 1) => 1059
   Natural FPS: 15.7
Loaded 1060 sonar frames i

In [ ]:
# Debug: Check for large distance jumps in the results
import numpy as np

# Calculate frame-to-frame distance changes
valid_distances = distance_results[distance_results['detection_success']]['distance_pixels'].dropna()
distance_changes = np.abs(np.diff(valid_distances))

# Find large jumps
threshold = 20  # Same as max_distance_change_pixels
large_jumps = distance_changes > threshold
large_jump_indices = np.where(large_jumps)[0]

print(f"Distance Stability Analysis:")
print(f"Total valid distance measurements: {len(valid_distances)}")
print(f"Large distance jumps (>{threshold} pixels): {np.sum(large_jumps)}")
print(f"Percentage of large jumps: {np.sum(large_jumps)/len(distance_changes)*100:.1f}%")

if len(large_jump_indices) > 0:
    print(f"\nLargest jumps found:")
    # Show the 5 largest jumps
    largest_indices = large_jump_indices[np.argsort(distance_changes[large_jump_indices])[-5:]]
    for idx in largest_indices:
        prev_dist = valid_distances.iloc[idx]
        next_dist = valid_distances.iloc[idx+1]
        change = distance_changes[idx]
        print(f"  Frame {idx} → {idx+1}: {prev_dist:.1f} → {next_dist:.1f} pixels (change: {change:.1f})")

# Check the actual distance values
print(f"\nDistance statistics:")
print(f"Mean: {valid_distances.mean():.1f} pixels")
print(f"Std: {valid_distances.std():.1f} pixels")
print(f"Min: {valid_distances.min():.1f} pixels") 
print(f"Max: {valid_distances.max():.1f} pixels")
print(f"Range: {valid_distances.max() - valid_distances.min():.1f} pixels")

In [ ]:
# Let's create a debug version of the processor to see what's happening
import utils.sonar_image_analysis as iau
import importlib
importlib.reload(iau)

# Create a debug processor that prints distance threshold decisions
class DebugSonarDataProcessor(iau.SonarDataProcessor):
    def analyze_frame(self, frame_u8, extent=None):
        result = super().analyze_frame(frame_u8, extent)
        
        # Print debug info when distance changes
        if result.distance_pixels is not None and self.previous_distance_pixels is not None:
            change = abs(result.distance_pixels - self.previous_distance_pixels)
            threshold = self.img_config.get('max_distance_change_pixels', 20)
            
            if change > threshold:
                print(f"🚨 LARGE DISTANCE CHANGE DETECTED:")
                print(f"   Previous: {self.previous_distance_pixels:.1f} pixels")
                print(f"   Current:  {result.distance_pixels:.1f} pixels")
                print(f"   Change:   {change:.1f} pixels (threshold: {threshold})")
                print(f"   Action:   {'SMOOTHED' if change > threshold else 'ACCEPTED'}")
        
        return result

# Test with a few frames to see if threshold is working
debug_processor = DebugSonarDataProcessor()

# Load a few frames and process them
cones, ts, extent, meta = iau.load_cone_run_npz(selected)
print(f"Testing distance threshold with {min(10, len(cones))} frames...")

for i in range(min(10, len(cones))):
    frame_u8 = iau.to_uint8_gray(cones[i])
    result = debug_processor.analyze_frame(frame_u8, extent)
    
    if result.detection_success:
        print(f"Frame {i}: distance = {result.distance_pixels:.1f} pixels")